In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-122101",
                  subscription_id='374bdf1a-c648-4244-a317-f0d1ef4b85c7',
                   resource_group='aml-quickstarts-122101'
                  )
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-122101
Azure region: southcentralus
Subscription id: 374bdf1a-c648-4244-a317-f0d1ef4b85c7
Resource group: aml-quickstarts-122101


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = 'my-cluster'

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found one, use it!')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                          max_nodes=4)
    
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found one, use it!
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    'C': uniform(0.1,100)
}
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory='.',
    entry_script = 'train.py',
    compute_target='my-cluster'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    policy=policy,
    estimator=est,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10
)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
best_run.get_details()

{'runId': 'HD_33fd7c16-63ac-4cae-8690-3779b7882299_9',
 'target': 'my-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-10-22T01:35:40.637158Z',
 'endTimeUtc': '2020-10-22T01:36:48.783889Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '391af68a-a9dc-4a78-848a-906645f0941d',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': [],
  'useAbsolutePath': False,
  'arguments': ['--C', '16.001859705013082'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'my-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'priority': None,
  'environment': {'name': 'Experiment udacity-project Environment',
   'version': 'Autosave_2020-10-22T01:23:38Z_f762fd4f',
  

In [12]:
!ls

outputs  README.md  training  train.py	udacity-project.ipynb


In [20]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
parameters = best_run.get_details()['runDefinition']['arguments']
# best_run.get_output()[1]
joblib.dump(parameters, filename='outputs/hyperdrive.joblib')

['outputs/hyperdrive.joblib']

In [21]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [24]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [51]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)

In [53]:
x_train.columns = x.columns
y_train.columns = ['y']

In [54]:
import pandas as pd
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=pd.concat([x_train,y_train], axis=1),
    label_column_name='y',
    n_cross_validations=5)

In [57]:
# Submit your automl run
auto_exp = Experiment(ws, 'automl-exp')
local_run = auto_exp.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_b82cf587-d6bd-44f0-b167-be3cec660c06

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [59]:
# Retrieve and save your best automl model.
best_auto_run, fitted_model = local_run.get_output()

joblib.dump(fitted_model, filename='outputs/automl.joblib')

['outputs/automl.joblib']

In [60]:
# Deleting cluster
cpu_cluster.delete()